In [1]:
import re
import os
import sys
import json
import gzip
import codecs

from time import sleep
from itertools import chain
from operator import itemgetter

from multiprocessing.dummy import Pool, Queue

import requests
import pandas as pd

from lxml import etree, html as lhtml
from tqdm.notebook import tqdm

In [2]:
PROXY = {'http': '3.141.186.75:3128', 'https': '3.141.186.75:3128', }
PROXY = {}

In [3]:
documents = []
page_num = 0
max_num_documents = 300

with tqdm(desc='Number of documents', total=max_num_documents) as pbar:
    while len(documents) < max_num_documents:
        page_num += 1
        page = requests.get('https://gg.deals/games/',
                            params={'sort': 'metascore', 'view': 'list', 'page': page_num},
                            proxies=PROXY)

        tree = lhtml.fromstring(page.content)

        documents_ = tree.xpath('//div[@class="list-items"]//a[@class="game-hoverable full-link"]/@href')
        documents_ = filter(lambda s: s.startswith('/game/'), documents_)
        documents_ = map(lambda s: 'https://gg.deals' + s, documents_)
        documents_ = list(documents_)

        documents.extend(documents_)
        pbar.update(len(documents_))

    documents = documents[:max_num_documents]

len(documents)

300

In [4]:
HEADERS_API = '''
accept: application/json, text/javascript, */*; q=0.01
accept-language: ru-RU,ru;q=0.9
cache-control: no-cache
pragma: no-cache
sec-ch-ua: "Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"
sec-ch-ua-mobile: ?0
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: same-origin
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36
x-requested-with: XMLHttpRequest
'''

HEADERS_API = HEADERS_API.strip().split('\n')
HEADERS_API = [h.split(': ') for h in HEADERS_API]
HEADERS_API = dict(HEADERS_API)
HEADERS_API

{'accept': 'application/json, text/javascript, */*; q=0.01',
 'accept-language': 'ru-RU,ru;q=0.9',
 'cache-control': 'no-cache',
 'pragma': 'no-cache',
 'sec-ch-ua': '"Chromium";v="88", "Google Chrome";v="88", ";Not A Brand";v="99"',
 'sec-ch-ua-mobile': '?0',
 'sec-fetch-dest': 'empty',
 'sec-fetch-mode': 'cors',
 'sec-fetch-site': 'same-origin',
 'user-agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.182 Safari/537.36',
 'x-requested-with': 'XMLHttpRequest'}

In [5]:
HEADERS_DLC = '''
accept: */*
accept-encoding: gzip, deflate, br
accept-language: ru,en-US;q=0.9,en;q=0.8,ru-RU;q=0.7,bg;q=0.6
cache-control: no-cache
content-length: 100
content-type: application/x-www-form-urlencoded; charset=UTF-8
cookie: gg-session=95qt38h375rfkkqig5g8m8or0t; firstVisit=1617642052; gg_csrf=1f935b6aa3417d2a0274f9ca3aaeaf6f29ad1debs%3A88%3A%22WHJtNk5rRVQxZDI1V1N4WWQ5NVMySHNFVUZfRXJyUXDLBVxvItaDhqvCp8mENFWpzTxs-hK3-4nRzHQmFxSpzA%3D%3D%22%3B; _ga=GA1.2.230921141.1617642056; _hjid=9a7da22e-f425-4e23-9abb-5aa9dc2f4ab4; ab_f29620a0=894835718; _gid=GA1.2.654496498.1619121776; _hjIncludedInSessionSample=0; _hjAbsoluteSessionInProgress=0; ab_df245b48=2333513272; _gat_UA-36788988-8=1; _gat_testAB_df245b48=1; _gat_testAB_f29620a0=1
origin: https://gg.deals
pragma: no-cache
sec-ch-ua: "Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"
sec-ch-ua-mobile: ?0
sec-fetch-dest: empty
sec-fetch-mode: cors
sec-fetch-site: same-origin
user-agent: Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_6) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/89.0.4389.114 Safari/537.36
x-requested-with: XMLHttpRequest
'''

HEADERS_DLC = HEADERS_DLC.strip().split('\n')
HEADERS_DLC = [h.split(': ') for h in HEADERS_DLC]
HEADERS_DLC = dict(HEADERS_DLC)
HEADERS_DLC

{'accept': '*/*',
 'accept-encoding': 'gzip, deflate, br',
 'accept-language': 'ru,en-US;q=0.9,en;q=0.8,ru-RU;q=0.7,bg;q=0.6',
 'cache-control': 'no-cache',
 'content-length': '100',
 'content-type': 'application/x-www-form-urlencoded; charset=UTF-8',
 'cookie': 'gg-session=95qt38h375rfkkqig5g8m8or0t; firstVisit=1617642052; gg_csrf=1f935b6aa3417d2a0274f9ca3aaeaf6f29ad1debs%3A88%3A%22WHJtNk5rRVQxZDI1V1N4WWQ5NVMySHNFVUZfRXJyUXDLBVxvItaDhqvCp8mENFWpzTxs-hK3-4nRzHQmFxSpzA%3D%3D%22%3B; _ga=GA1.2.230921141.1617642056; _hjid=9a7da22e-f425-4e23-9abb-5aa9dc2f4ab4; ab_f29620a0=894835718; _gid=GA1.2.654496498.1619121776; _hjIncludedInSessionSample=0; _hjAbsoluteSessionInProgress=0; ab_df245b48=2333513272; _gat_UA-36788988-8=1; _gat_testAB_df245b48=1; _gat_testAB_f29620a0=1',
 'origin': 'https://gg.deals',
 'pragma': 'no-cache',
 'referer': 'https://gg.deals/game/tricky-towers/',
 'sec-ch-ua': '"Google Chrome";v="89", "Chromium";v="89", ";Not A Brand";v="99"',
 'sec-ch-ua-mobile': '?0',
 'sec-fetc

In [6]:
def get_history(app_id, page_url):
    headers = HEADERS_API.copy()
    headers['referer'] = page_url
    
    page_url = f'https://gg.deals/ru/games/chartHistoricalData/{app_id}/'
    page = requests.get(page_url, params={'hideKeyshops': 0},
                        headers=headers, proxies=PROXY)
    
    if not page.ok:
        print('Failed to get page:', page_url)
        return False, []
    
    data = page.json()

    if data['code']:
        print('Error code for page:', page_url, file=sys.stderr)
        return False, []
    
    if 'chartData' not in data:
        print('No chartData for page:', page_url, file=sys.stderr)
        return False, []
    
    data = data['chartData']
    
    if 'deals' not in data:
        print('No deals for page:', page_url, file=sys.stderr)
        return False, []
    
    data = data['deals']

    history = [{'ts': r['x'] // 1000, 'price': r['y'], 'shop': r['shop']} for r in data]
    return True, history


def get_dlc_pack_info(game_id, page_url, type_):
    headers = HEADERS_DLC.copy()
    headers['referer'] = page_url
    
    page = requests.post(
        f'https://gg.deals/ru/games/relations/{game_id}/', 
        params={'type': type_, 'offset': 0, 'hideKeyshops': 0},
        data='gg_csrf=WHJtNk5rRVQxZDI1V1N4WWQ5NVMySHNFVUZfRXJyUXDLBVxvItaDhqvCp8mENFWpzTxs-hK3-4nRzHQmFxSpzA%3D%3D',
        headers=HEADERS_DLC,
    )
    tree = lhtml.fromstring(page.content)
    
    dlcs = tree.xpath('//a[@class="full-link"]/@href')
    dlcs = map(lambda s: 'https://gg.deals' + s, dlcs)
    dlcs = list(dlcs)
    
    return dlcs


def extract_page_info(tree):
    info = {}
    
    meta = tree.xpath('//div[@class="game-info-content"]')[0]
    
    name = meta.xpath('//ul[@class="breadcrumbs-list"]//a/span[@itemprop="name"]/text()')
    name = name[-1]
    info['name'] = name
    
    image = meta.xpath('//div[@class="game-info-image"]//img')[0].get('src')
    info['image'] = image
    
    release_date = meta.xpath('.//div[contains(@class, "game-info-details-section-release")]/p/text()')
    release_date = ''.join(release_date)
    info['release_date'] = release_date
    
    developer = meta.xpath('.//div[contains(@class, "game-info-details-section-developer")]/p/text()')
    developer = ''.join(developer)
    info['developer'] = developer
    
    metacritic_score = meta.xpath('.//a[contains(@class, "score-metascore")]//span[@class="overlay"]/text()')
    metacritic_score = ''.join(metacritic_score)
    if metacritic_score:
        info['metacritic_score'] = int(metacritic_score)
        
    user_score = meta.xpath('.//a[contains(@class, "score-userscore")]//span[@class="overlay"]/text()')
    user_score = ''.join(user_score)
    if user_score:
        info['user_score'] = float(user_score)
        
    reviews = meta.xpath('.//span[@class="reviews-label"]')
    if reviews:
        reviews = reviews[0]
        match = re.search(
            '(\d+)% of the (\d+(?:,\d+)*) user reviews for this game are positive.',
            reviews.attrib['title'],
        )
        
        info['review_label'] = reviews.text
        info['review_positive_pctg'] = int(match.group(1))
        info['review_count'] = int(re.sub('\D', '', match.group(2)))

    info['genres'] = meta.xpath('.//div[@id="game-info-genres"]//a[@class="badge"]/text()')
    info['tags'] = meta.xpath('.//div[@id="game-info-tags"]//a[@class="badge"]/text()')
    info['features'] = meta.xpath('.//div[@id="game-info-features"]//a[@class="badge"]/text()')

    info['pc_systems'] = tree.xpath('//div[@class="game-requirements-tabs"]//a[@role="button"]/text()')
    
    card_actions = meta.xpath('//div[@class="game-info-actions"]')
    if card_actions:
        card_actions = card_actions[0]
        
        wishlist_cnt = card_actions.xpath('.//div[contains(@class, "wishlisted-game")]'
                                          '//span[@class="count"]/text()')
        wishlist_cnt = int(wishlist_cnt[0])
        info['wishlist_count'] = wishlist_cnt
        
        alert_cnt = card_actions.xpath('.//div[contains(@class, "alerted-game")]'
                                       '//span[@class="count"]/text()')
        alert_cnt = int(alert_cnt[0])
        info['alert_count'] = alert_cnt
        
        owners_cnt = card_actions.xpath('.//div[contains(@class, "owned-game")]'
                                        '//span[@class="count"]/text()')
        owners_cnt = int(owners_cnt[0])
        info['owners_count'] = owners_cnt
    
    market_url = meta.xpath('//a[@class="game-link-widget"]/@href')
    if market_url:
        market_url = market_url[0]
        market_url = requests.get(market_url, proxies=PROXY).url
        info['market_url'] = market_url
    
    return info


def process_page(page_url):
    info = {
        'url':    page_url,
        'status': False,
    }
    
    page = requests.get(page_url, proxies=PROXY)
    tree = lhtml.fromstring(page.content)
    app_id = int(tree.xpath('.//div[@data-game-id]/@data-game-id')[0])

    info.update(extract_page_info(tree))

    try:
        info['dlcs'] = get_dlc_pack_info(app_id, page_url, 'dlc')
    except:
        info['dlcs'] = []

    try:
        info['packs'] = get_dlc_pack_info(app_id, page_url, 'packs')
    except:
        info['packs'] = []
    
    is_ok, history = get_history(app_id, page_url)
    info['price_history'] = history
    info['status'] = is_ok
    
    return info

In [14]:
%%time

print(json.dumps(
    process_page('https://gg.deals/game/tricky-towers/'),
    indent=4,
))

{
    "url": "https://gg.deals/game/tricky-towers/",
    "status": true,
    "name": "Tricky Towers",
    "image": "https://img.gg.deals/70/ae/26c780669e69d3cd65f6bcd9f5aefa6dfaa6_307xt176.jpg",
    "release_date": "02 Aug 2016",
    "developer": "WeirdBeard",
    "metacritic_score": 80,
    "user_score": 6.8,
    "review_label": "Very Positive",
    "review_positive_pctg": 90,
    "review_count": 6630,
    "genres": [
        "Casual",
        "Indie"
    ],
    "tags": [
        "Party Game",
        "Casual",
        "Puzzle",
        "Physics",
        "Building",
        "Funny",
        "Family Friendly",
        "Multiplayer",
        "Cartoony",
        "Arcade",
        "Singleplayer",
        "Online Co-Op",
        "Local Multiplayer",
        "Local Co-Op",
        "4 Player Local",
        "PvP",
        "Strategy",
        "Party",
        "Colorful",
        "Stylized",
        "Indie"
    ],
    "features": [
        "Single-player",
        "Online PvP",
        "Share

In [8]:
queue = Queue()

for url in documents:
    queue.put(url)

In [9]:
import os, shutil

if os.path.isdir('data'):
    shutil.rmtree('data')
os.mkdir('data')

In [10]:
def process_page_wrapper(i):
    with gzip.open('data/part_{:05d}.jsonl.gz'.format(i), mode='wb') as f_json:
        f_json = codecs.getwriter('utf8')(f_json)

        while not queue.empty():
            is_ok, reason = True, None
            url = queue.get()

            try:
                record = process_page(url)
                if record is not None:
                    record_str = json.dumps(record, ensure_ascii=False)
                    print(record_str, file=f_json)
                else:
                    is_ok, reason = False, 'Failed to get url'
            except Exception as e:
                is_ok, reason = False, e
                
                if isinstance(e, KeyboardInterrupt):
                    break
            finally:
                with lock:
                    if not is_ok:
                        print('Failed:', url, file=sys.stderr)
                        print('Reason:', reason, file=sys.stderr)
                    pbar.update(1)

In [11]:
with Pool(processes=8) as pool, tqdm(total=queue.qsize()) as pbar:
    lock = pbar.get_lock()
    pool.map(process_page_wrapper, range(pool._processes))

Failed: https://gg.deals/game/company-of-heroes-legacy-edition/
Reason: HTTPSConnectionPool(host='gg.deals', port=443): Max retries exceeded with url: /ru/games/relations/71475/?type=dlc&offset=0&hideKeyshops=0 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11bf7e2e8>: Failed to establish a new connection: [Errno 60] Operation timed out'))
Failed: https://gg.deals/game/sid-meier-s-civilization-iii-complete/
Reason: HTTPSConnectionPool(host='gg.deals', port=443): Max retries exceeded with url: /game/sid-meier-s-civilization-iii-complete/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11bc2e278>: Failed to establish a new connection: [Errno 60] Operation timed out'))
Failed: https://gg.deals/game/inside/
Reason: HTTPSConnectionPool(host='gg.deals', port=443): Max retries exceeded with url: /ru/games/chartHistoricalData/21495/?hideKeyshops=0 (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConne

Failed: https://gg.deals/game/europa-universalis-iv/
Reason: HTTPSConnectionPool(host='store.steampowered.com', port=443): Max retries exceeded with url: /app/236850/ (Caused by NewConnectionError('<urllib3.connection.VerifiedHTTPSConnection object at 0x11bda2978>: Failed to establish a new connection: [Errno 60] Operation timed out'))


In [12]:
!gzcat data/part_00000.jsonl.gz | head -n5 >example.jsonl

gzcat: error writing to output: Broken pipe
gzcat: data/part_00000.jsonl.gz: uncompress failed
